In [231]:
#!/usr/bin/env python3

# from blind_rename import rename_imposter_cols

from cimaq_utils import get_cimaq_dir_paths
from sniffbytes import flatten
from sniffbytes import loadfiles
from sniffbytes import loadimages
from cimaq_utils import repair_enc_task
from cimaq_utils import fetch_cimaq

from json_read import json_read
# from sniffbytes import sortmap

from sniffbytes import *

# from zipctl import uzipfiles
# from zipctl import getnamelist
# from zipctl import getinfolist

import csv
import json
import nilearn
import numpy as np
import struct
from chardet import UniversalDetector as udet
from io import StringIO
from os.path import expanduser as xpu
from os import listdir as ls
from os.path import join as pjoin
from typing import Union
from zipfile import ZipFile
from tqdm import tqdm

from removeEmptyFolders import removeEmptyFolders
from multiple_replace import multiple_replace
cimaq_dir = '~/../../media/francois/seagate_1tb/cimaq_03-19/cimaq_derivatives'
otherdir = '/home/fnadeau/projects/rrg-pbellec/fnadeau/cimaq_03-19/cimaq_03-19/sub-127228/ses-4/anat'
dlst = get_cimaq_dir_paths(cimaq_dir)[0]#[['confdir', 'events_dir']]


In [232]:
cimaq = fetch_cimaq(cimaq_dir)

sniffing: 100%|██████████| 94/94 [00:03<00:00, 31.17it/s]
repairing: 284it [00:01, 259.85it/s]
saving: 284it [00:00, 3176.22it/s]
encoding_task: 94it [00:01, 75.64it/s]


In [233]:
cimaq = cimaq.sort_values('dccid')
cimaq

,subid,pscid,dccid,stereonl,behavioral,confounds,events,func
26,sub-4509950-108391,4509950,108391,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...
62,sub-7424803-120839,7424803,120839,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...
73,sub-7874568-122922,7874568,122922,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...
14,sub-3865361-127228,3865361,127228,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...
43,sub-5760364-139593,5760364,139593,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...
...,...,...,...,...,...,...,...,...
66,sub-7516889-979001,7516889,979001,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...
15,sub-3874544-983291,3874544,983291,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...
54,sub-6633412-988602,6633412,988602,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...
29,sub-4576408-996599,4576408,996599,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...,/home/francois/../../media/francois/seagate_1t...


In [234]:
allscans = pd.concat([loadfiles(loadimages(pjoin(dname(xpu(cimaq_dir)), sub))).T
                      for sub in [itm for itm in ls(dname(xpu(cimaq_dir)))
                        if itm.startswith('sub-')]], axis = 1).T
allscans[['dccid', 'modality', 'general']] = [(row[1].fname.split('_')[0].split('-')[1],
                                               row[1].fname.split('_')[-1],
                                               bname(dname(row[1].fpaths)))
                     for row in allscans.iterrows()]

# allscans = allscans.loc[[row[0] for row in allscans.iterrows()
#                          if str(row[1].dccid) in [str(ids) for ids
#                                                   in cimaq.dccid]]]
#.reset_index(drop = True).sort_values('fname')
# allparams = allscans.loc[[row[0] for row in allscans.iterrows()
#                           if row[1].ext != '.json']]
# allscans = allscans.loc[[row[0] for row in allparams.iterrows()
#                          if '.json' not in row[1].fpaths]]
test = dict((grp, dict((sgrp, allscans.groupby('dccid').get_group(grp).groupby('general').get_group(sgrp))
              for sgrp in allscans.groupby('dccid').get_group(grp).groupby('general').groups)) for
             grp in allscans.groupby('dccid').groups)



In [230]:
test2 = df.from_dict(test, orient = 'index')
test2

""


In [208]:
list(test2)

['anat', 'dwi', 'fmap', 'func']

In [204]:
test2.iloc[0].fmap

,fname,ext,fpaths,dccid,modality,general
14,sub-146090_ses-4_run-01_epi,.json,/home/francois/../../media/francois/seagate_1t...,146090,epi,fmap
15,sub-146090_ses-4_run-01_epi,.nii.gz,/home/francois/../../media/francois/seagate_1t...,146090,epi,fmap
16,sub-146090_ses-4_run-01_magnitude1,.nii.gz,/home/francois/../../media/francois/seagate_1t...,146090,magnitude1,fmap
17,sub-146090_ses-4_run-01_magnitude1,.json,/home/francois/../../media/francois/seagate_1t...,146090,magnitude1,fmap
18,sub-146090_ses-4_run-01_magnitude2,.nii.gz,/home/francois/../../media/francois/seagate_1t...,146090,magnitude2,fmap
19,sub-146090_ses-4_run-01_magnitude2,.json,/home/francois/../../media/francois/seagate_1t...,146090,magnitude2,fmap
20,sub-146090_ses-4_run-01_phasediff,.json,/home/francois/../../media/francois/seagate_1t...,146090,phasediff,fmap
21,sub-146090_ses-4_run-01_phasediff,.nii.gz,/home/francois/../../media/francois/seagate_1t...,146090,phasediff,fmap
22,sub-146090_ses-4_run-02_epi,.json,/home/francois/../../media/francois/seagate_1t...,146090,epi,fmap
23,sub-146090_ses-4_run-02_epi,.nii.gz,/home/francois/../../media/francois/seagate_1t...,146090,epi,fmap


In [181]:
# params = [pd.concat([row[1][['fname']], df.from_dict(json_read(row[1].fpaths, 'r'), orient = 'index')])
#           for row in test[0].iterrows()
#           if row[1].ext == '.json']
# # allparams = pd.concat(((itm.groupby('fname').get_group(grp)
# #                  for grp in itm.groupby('fname').groups)
# #              for itm in allparams), axis = 1)
# # pd.concat(params, axis =1).T.sort_values('fname')
# pd.concat(params, axis = 1).T.set_index('fname')

In [ ]:
def _update_timestamp(path: os.PathLike, set_new: bool) -> None:
    """
    Context manager to set the timestamp of the path to plus or
    minus a fixed delta, regardless of modifications within the context.

    if set_new is True, the delta is added. Otherwise, the delta is subtracted.
    """
    stats = os.stat(path)
    if set_new:
        new_timestamp = (stats.st_atime_ns + _TIMESTAMP_DELTA, stats.st_mtime_ns + _TIMESTAMP_DELTA)
    else:
        new_timestamp = (stats.st_atime_ns - _TIMESTAMP_DELTA, stats.st_mtime_ns - _TIMESTAMP_DELTA)
    try:
        yield
    finally:
        os.utime(path, ns=new_timestamp)


# Public Methods 